#### Import

In [3]:
import duckdb
con = duckdb.connect('data.db')

In [4]:
# show table
con.sql('show tables')

┌─────────────────────────────┐
│            name             │
│           varchar           │
├─────────────────────────────┤
│ Bus_Stop_Info               │
│ Business_Operation          │
│ MRT_Business_Area           │
│ MRT_Event                   │
│ MRT_Flow_Record             │
│ MRT_Station_Info            │
│ Representative              │
│ Ubike_Station_Info          │
│ Ubike_Station_Rental_Record │
│ Village_Info                │
│ Village_Population_By_Age   │
│ shop_rental_listing         │
├─────────────────────────────┤
│           12 rows           │
└─────────────────────────────┘

## 查詢周邊商圈、租房資訊

### 商圈平均租金以及周邊捷運站

In [ ]:
def get_organization_data(district = None):
    # 檢查使用者是否勾選 district，若有則根據選擇的區域回傳，否則回傳全部
    where_clause = f"WHERE district = '{district}'" if district else ""
    
    res = con.sql(f"""--sql
        WITH nearest_stations AS (
            SELECT 
                s.district,
                s.case_name,
                s.address,
                s.monthly_rent,
                s.area_ping,
                m.station_id,
                m.station_name,
                MIN(
                    6371 * ACOS(
                        COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                        COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                        SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
                    )
                ) AS nearest_distance_km
            FROM Shop_Rental_Listing s
            CROSS JOIN MRT_Station_Info m
            GROUP BY s.district, s.case_name, s.address, s.monthly_rent, s.area_ping, m.station_id, m.station_name
            HAVING nearest_distance_km <= 1
        )
        SELECT 
            mba.name , -- 商圈名稱
            ROUND(AVG(ns.monthly_rent)) AS average_monthly_rent,
            ns.station_name,
            mba.tag ,
            ns.district
        FROM nearest_stations ns
        JOIN MRT_Business_Area mba ON ns.station_id = mba.station_id -- 加入商圈數據
        {where_clause} -- 動態加入條件
        GROUP BY mba.name, mba.tag, ns.station_name, ns.district
        ORDER BY ns.district, ns.station_name; 
        """)
    
    return res

# 範例
get_organization_data(district = '中山區')
# get_organization_data()

┌────────────────────────┬──────────────────────┬──────────────┬──────────┬──────────┐
│          name          │ average_monthly_rent │ station_name │   tag    │ district │
│        varchar         │        double        │   varchar    │ varchar  │ varchar  │
├────────────────────────┼──────────────────────┼──────────────┼──────────┼──────────┤
│ 中山捷運站周邊         │             183482.0 │ 中山         │ 買在商圈 │ 中山區   │
│ 條通(商圈)             │             183482.0 │ 中山         │ 吃在商圈 │ 中山區   │
│ 晶華酒店、欣欣百貨周邊 │             183482.0 │ 中山         │ 玩在商圈 │ 中山區   │
│ 中山北路婚紗(商圈)     │             183482.0 │ 中山         │ 玩在商圈 │ 中山區   │
│ 朝陽服飾材料(商圈)     │             183482.0 │ 中山         │ 買在商圈 │ 中山區   │
│ 赤峰街 (商圈)          │             183482.0 │ 中山         │ 玩在商圈 │ 中山區   │
│ 中山國中捷運站周邊     │             172210.0 │ 中山國中     │ 吃在商圈 │ 中山區   │
│ 大直(商圈)             │             252600.0 │ 劍南路       │ 玩在商圈 │ 中山區   │
│ 南京復興捷運站周邊     │             268530.0 │ 南京復興     │ 吃在商圈 │ 中山區   │
│ 後站(商圈)            

In [15]:
def get_filtered_shop_rentals(district=None, min_rent=None, max_rent=None, min_area=None, max_area=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district is not None:
        conditions.append(f"s.district = '{district}'")
    if min_rent is not None:
        conditions.append(f"s.monthly_rent >= {min_rent}")
    if max_rent is not None:
        conditions.append(f"s.monthly_rent <= {max_rent}")
    if min_area is not None:
        conditions.append(f"s.area_ping >= {min_area}")
    if max_area is not None:
        conditions.append(f"s.area_ping <= {max_area}")
    
    # 將條件組合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    # 執行 SQL 查詢
    query = f"""--sql
    SELECT 
        s.district,
        s.case_name,
        s.address, 
        s.monthly_rent, 
        ROUND(s.monthly_rent / s.area_ping) AS monthly_rent_per_ping, 
        s.area_ping, 
        s.shop_floor, 
        s.total_floor,
        s.deposit, 
        r.name, 
        r.phone
    FROM Shop_Rental_Listing s
    LEFT JOIN Representative r ON s.phone = r.phone
    
    {where_clause} -- 動態加入條件
    """
    res = con.sql(query)
    df = res.df()
    return df

# 範例
get_filtered_shop_rentals(district='文山區', min_rent=10000, max_rent=50000, min_area=10, max_area=50)

,district,case_name,address,monthly_rent,monthly_rent_per_ping,area_ping,shop_floor,total_floor,deposit,name,phone
0,文山區,木柵中心黃金店面,台北市文山區木柵路三段,45000,1324.0,34,1,7,90000,申續強,0289786553
1,文山區,正羅斯福路挑高面寬金店辦,台北市文山區羅斯福路六段,45000,1286.0,35,1,7,90000,范國晨,0289786583
2,文山區,羅斯福路小店面,台北市文山區羅斯福路五段,33000,2357.0,14,1,7,66000,廖佑霆,0289787023
3,文山區,動物園旁黃金店面出租,台北市文山區新光路一段,33800,1090.0,31,1,5,67600,楊善全,0289787026
4,文山區,辛亥路複合式金店面,台北市文山區辛亥路六段,40000,1000.0,40,1~2,4,80000,全民房仲｜潘小榮,0970883312
5,文山區,羅斯福黃金店面,台北市文山區羅斯福路六段,49000,4083.0,12,1~2,2,98000,廖佑霆,0289787023
6,文山區,碩園黃金店面,台北市文山區景興路,48000,1263.0,38,1,13,96000,廖佑霆,0289787023


: 

In [2]:
def get_filtered_shop_rentals(min_rent=None, max_rent=None, min_area=None, max_area=None, trade_area_name=None):
    # 动态构建 WHERE 条件
    conditions = []
    
    # 根据租金范围加入条件
    if min_rent is not None:
        conditions.append(f"s.monthly_rent >= {min_rent}")
    if max_rent is not None:
        conditions.append(f"s.monthly_rent <= {max_rent}")
    
    # 根据面积范围加入条件
    if min_area is not None:
        conditions.append(f"s.area_ping >= {min_area}")
    if max_area is not None:
        conditions.append(f"s.area_ping <= {max_area}")
    
    # 根据商圈名称加入条件
    if trade_area_name is not None:
        conditions.append(f"mba.name = '{trade_area_name}'")
    
    # 将条件组合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    # 执行 SQL 查询
    query = f"""--sql
    SELECT 
        s.address, 
        s.monthly_rent, 
        ROUND(s.monthly_rent / s.area_ping) AS monthly_rent_per_ping, 
        s.area_ping, 
        s.shop_floor, 
        s.total_floor,
        s.deposit, 
        r.name, 
        r.phone
    FROM Shop_Rental_Listing s
    LEFT JOIN Representative r ON s.phone = r.phone
    JOIN MRT_Business_Area mba ON s.district = mba.district -- 连接商圈数据
    {where_clause} -- 动态加入条件
    """
    res = con.sql(query)
    return res
get_filtered_shop_rentals(min_rent=10000, max_rent=30000, min_area=10, max_area=50, trade_area_name="臺大公館(商圈)")

NameError: name 'con' is not defined

## 店面資料與所在商圈及鄰近捷運站

In [7]:
con.sql("""--sql
    WITH distances AS (
    SELECT
        s.case_id,
        s.district,
        s.village,
        s.case_name,
        s.monthly_rent,
        s.area_ping,
        s.address,
        m.station_id,
        m.station_name,
        (
            6371 * ACOS(
                COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
            )
        ) AS distance_km
    FROM Shop_Rental_Listing s
    CROSS JOIN MRT_Station_Info m
),
nearest_stations AS (
    SELECT
        d.case_id, 
        d.district,
        d.case_name,
        d.address,
        d.village,
        d.station_id,
        d.station_name,
        d.monthly_rent,
        d.area_ping,
        MIN(d.distance_km) AS nearest_distance_km
    FROM distances d
    GROUP BY d.case_id, d.district, d.case_name, d.address, d.village, d.station_id, d.station_name, d.monthly_rent, d.area_ping
    HAVING MIN(d.distance_km) <= 1
)
SELECT 
    ns.case_id,
    ns.district,
    ns.village,
    ns.address,
    ns.case_name,
    ns.station_id,
    ns.station_name,
    ns.monthly_rent,
    ns.area_ping,
    mba.name, -- 商圈名稱
    mba.tag
FROM nearest_stations ns
JOIN MRT_Business_Area mba ON ns.station_id = mba.station_id -- 加入商圈數據d.
ORDER BY ns.case_id, ns.village, ns.nearest_distance_km ASC;
""")

┌─────────┬──────────┬─────────┬──────────────────────────┬──────────────────────────────────────────────┬────────────┬──────────────┬──────────────┬───────────┬──────────────────────┬──────────┐
│ case_id │ district │ village │         address          │                  case_name                   │ station_id │ station_name │ monthly_rent │ area_ping │         name         │   tag    │
│  int64  │ varchar  │ varchar │         varchar          │                   varchar                    │  varchar   │   varchar    │    int32     │   int32   │       varchar        │ varchar  │
├─────────┼──────────┼─────────┼──────────────────────────┼──────────────────────────────────────────────┼────────────┼──────────────┼──────────────┼───────────┼──────────────────────┼──────────┤
│       1 │ 士林區   │ 仁勇里  │ 台北市士林區文林路       │ 士林夜市TOD1+2樓店面                         │ R16        │ 士林         │        84000 │        19 │ 承德路中古汽車(商圈) │ 買在商圈 │
│       1 │ 士林區   │ 仁勇里  │ 台北市士林區文林路       │ 士林夜市TOD1+2樓店

In [81]:
def get_shop_flow_data(case_name=None, business_area_name=None):
    # 動態構建 WHERE 條件
    conditions = []
    if case_name:
        conditions.append(f"cf.case_name = '{case_name}'")
    if business_area_name:
        conditions.append(f"bai.business_area_name = '{business_area_name}'")
    
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
    WITH mrt_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            m.station_id AS mrt_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(m.latitude)) *
                    COS(RADIANS(m.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(m.latitude))
                )
            ) AS mrt_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN MRT_Station_Info m
    ),
    ubike_distances AS (
        SELECT
            s.case_id,
            s.district,
            s.village,
            s.case_name,
            u.station_id AS ubike_station_id,
            (
                6371 * ACOS(
                    COS(RADIANS(s.latitude)) * COS(RADIANS(u.latitude)) *
                    COS(RADIANS(u.longitude) - RADIANS(s.longitude)) +
                    SIN(RADIANS(s.latitude)) * SIN(RADIANS(u.latitude))
                )
            ) AS ubike_distance_km
        FROM Shop_Rental_Listing s
        CROSS JOIN Ubike_Station_Info u
    ),
    mrt_nearest_stations AS (
        SELECT 
            d.case_id,
            d.district,
            d.case_name,
            d.village,
            d.mrt_station_id
        FROM mrt_distances d
        WHERE d.mrt_distance_km <= 1
    ),
    ubike_nearest_stations AS (
        SELECT 
            d.case_id,
            d.district,
            d.case_name,
            d.village,
            d.ubike_station_id
        FROM ubike_distances d
        WHERE d.ubike_distance_km <= 1
    ),
    -- 不在此處過濾時段，使 MRT flow 保留所有時段
    mrt_flow_data AS (
        SELECT
            mf.station_id AS mrt_station_id, 
            mf.time_period,
            ROUND(AVG(mf.entrance_count + mf.exit_count)) AS avg_mrt_flow
        FROM MRT_Flow_Record mf
        WHERE mf.date >= CURRENT_DATE - INTERVAL '2 years' AND time_period NOT BETWEEN 2 AND 5
        GROUP BY mf.station_id, mf.time_period
    ),
    ubike_flow_data AS (
        SELECT
            uf.station_id AS ubike_station_id,
            uf.time_period,
            ROUND(AVG(uf.rent_count + uf.return_count)) AS avg_ubike_flow
        FROM Ubike_Station_Rental_Record uf
        WHERE uf.date >= CURRENT_DATE - INTERVAL '2 years'
        GROUP BY uf.station_id, uf.time_period
    ),
    -- 將 mrt_nearest_stations 與 mrt_flow_data JOIN，彙整出以 case_id 為單位的 MRT flow 資料
    mrt_case_flow AS (
        SELECT
            mrt.case_id,
            mrt.district,
            mrt.case_name,
            mrt.village,
            mf.time_period,
            AVG(mf.avg_mrt_flow) AS avg_mrt_flow
        FROM mrt_nearest_stations mrt
        JOIN mrt_flow_data mf ON mf.mrt_station_id = mrt.mrt_station_id
        GROUP BY mrt.case_id, mrt.district, mrt.case_name, mrt.village, mf.time_period
    ),
    -- 將 ubike_nearest_stations 與 ubike_flow_data JOIN，彙整出以 case_id 為單位的 Ubike flow 資料
    ubike_case_flow AS (
        SELECT
            ubike.case_id,
            ubike.district,
            ubike.case_name,
            ubike.village,
            uf.time_period,
            AVG(uf.avg_ubike_flow) AS avg_ubike_flow
        FROM ubike_nearest_stations ubike
        JOIN ubike_flow_data uf ON uf.ubike_station_id = ubike.ubike_station_id
        GROUP BY ubike.case_id, ubike.district, ubike.case_name, ubike.village, uf.time_period
    ),
    -- FULL JOIN 將MRT與Ubike的case flow合併，確保沒有MRT資料的時段依然會出現
    combined_flow AS (
        SELECT
            COALESCE(mcf.case_id, ucf.case_id) AS case_id,
            COALESCE(mcf.district, ucf.district) AS district,
            COALESCE(mcf.case_name, ucf.case_name) AS case_name,
            COALESCE(mcf.village, ucf.village) AS village,
            COALESCE(mcf.time_period, ucf.time_period) AS time_period,
            ROUND(COALESCE(mcf.avg_mrt_flow,0) + COALESCE(ucf.avg_ubike_flow,0)) AS avg_total_flow
        FROM mrt_case_flow mcf
        FULL JOIN ubike_case_flow ucf 
            ON mcf.case_id = ucf.case_id
           AND mcf.time_period = ucf.time_period
    ),
    business_area_info AS (
        SELECT 
            ns.case_id,
            mba.name AS business_area_name,
            mba.tag AS business_area_tag
        FROM mrt_nearest_stations ns
        JOIN MRT_Business_Area mba ON ns.mrt_station_id = mba.station_id
        GROUP BY ns.case_id, mba.name, mba.tag
    )
    SELECT 
        cf.case_id,
        cf.district,
        cf.village,
        cf.case_name,
        bai.business_area_name,
        cf.time_period,
        cf.avg_total_flow
    FROM combined_flow cf
    LEFT JOIN business_area_info bai ON cf.case_id = bai.case_id
    {where_clause}
    ORDER BY cf.district, cf.village, cf.case_name, bai.business_area_name, cf.time_period ASC;
    """)

    return res

# 範例呼叫
get_shop_flow_data(case_name='正隆官邸優質店面一樓', business_area_name='永康(商圈)')
# get_shop_flow_data(case_name='正隆官邸優質店面一樓')
# get_shop_flow_data()


┌─────────┬──────────┬─────────┬──────────────────────┬────────────────────┬─────────────┬────────────────┐
│ case_id │ district │ village │      case_name       │ business_area_name │ time_period │ avg_total_flow │
│  int64  │ varchar  │ varchar │       varchar        │      varchar       │    int64    │     double     │
├─────────┼──────────┼─────────┼──────────────────────┼────────────────────┼─────────────┼────────────────┤
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           0 │          260.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           1 │           17.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           2 │            3.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           3 │            2.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           4 │            3.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 永康(商圈)         │           5 │            3.0 │
│     254 │ 中正區   │ 文北里  │ 正隆官邸優質店面一樓 │ 

## 村裡人口、年齡、性別

In [ ]:
def get_village_data(district=None, village=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district:
        conditions.append(f"vi.district = '{district}'")
    if village:
        conditions.append(f"vi.village = '{village}'")
    
    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    res = con.sql(f"""--sql
        SELECT vi.district, vi.village, vi.household_count, vi.avg_income,
        ROUND(AVG(vi.avg_income) OVER (PARTITION BY vi.district)) AS nearby_avg_income, vi.median_income,
        ROUND(AVG(vi.median_income) OVER (PARTITION BY vi.district)) AS nearby_median_income,
        ROUND(vi.male_population * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS male_population_ratio, 
        ROUND(vi.female_population * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS female_population_ratio, 
        ROUND(v.age_0_9 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_0_9_ratio, 
        ROUND(v.age_10_19 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_10_19_ratio, 
        ROUND(v.age_20_29 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_20_29_ratio, 
        ROUND(v.age_30_64 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_30_64_ratio, 
        ROUND(v.age_over_65 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.village), 4) AS avg_over_65_ratio,
        ROUND(v.age_0_9 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_0_9_ratio, 
        ROUND(v.age_10_19 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_10_19_ratio, 
        ROUND(v.age_20_29 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_20_29_ratio, 
        ROUND(v.age_30_64 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_30_64_ratio, 
        ROUND(v.age_over_65 * 1.0 / SUM(vi.male_population + vi.female_population) OVER (PARTITION BY vi.district), 4) AS nearby_over_65_ratio 
        FROM Village_Info vi
        LEFT JOIN Village_Population_By_Age v ON vi.district = v.district AND vi.village = v.village
        {where_clause}
    """)
    
    return res

# 範例
get_village_data(district = '士林區', village = '承德里')
# get_village_data(district = '士林區')
# get_village_data()

┌──────────┬─────────┬─────────────────┬────────────┬───────────────────┬───────────────┬──────────────────────┬───────────────────────┬─────────────────────────┬───────────────┬─────────────────┬─────────────────┬─────────────────┬───────────────────┬──────────────────┬────────────────────┬────────────────────┬────────────────────┬──────────────────────┐
│ district │ village │ household_count │ avg_income │ nearby_avg_income │ median_income │ nearby_median_income │ male_population_ratio │ female_population_ratio │ avg_0_9_ratio │ avg_10_19_ratio │ avg_20_29_ratio │ avg_30_64_ratio │ avg_over_65_ratio │ nearby_0_9_ratio │ nearby_10_19_ratio │ nearby_20_29_ratio │ nearby_30_64_ratio │ nearby_over_65_ratio │
│ varchar  │ varchar │      int64      │   int64    │      double       │     int64     │        double        │        double         │         double          │    double     │     double      │     double      │     double      │      double       │      double      │       double

## 競爭市場

In [88]:
def get_competitive_data(district=None, village=None, type=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district:
        conditions.append(f"district = '{district}'")
    if village:
        conditions.append(f"village = '{village}'")
    if type:
        conditions.append(f"business_type = '{type}'")
    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
            SELECT district, village, business_type, business_sub_type, COUNT(business_name) as shop_cnt, ROUND(AVG(capital)) as avg_capital
            FROM Business_Operation
            {where_clause}
            GROUP BY district, village, business_type, business_sub_type
          """)
    
    return res

get_village_data(district = '大安區', village = '民輝里')
# get_village_data(district = '大安區')
# get_village_data()

┌──────────┬─────────┬────────────────────────┬──────────────────────────────┬──────────┬─────────────┐
│ district │ village │     business_type      │      business_sub_type       │ shop_cnt │ avg_capital │
│ varchar  │ varchar │        varchar         │           varchar            │  int64   │   double    │
├──────────┼─────────┼────────────────────────┼──────────────────────────────┼──────────┼─────────────┤
│ 大安區   │ 民輝里  │ 批發及零售業           │ 布疋及服飾品零售業           │        2 │   1100000.0 │
│ 大安區   │ 民輝里  │ 住宿及餐飲業           │ 餐食業                       │        6 │    126833.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 食品、飲料及菸草製品批發業   │        1 │      3000.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 汽機車及其零配件、用品零售業 │        2 │   1482500.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 管理顧問業                   │        2 │     75000.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 專門設計業                   │        3 │    516667.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 其他專業、科學及技術服務業   │        2 │    225000.0 │
│ 大安區   │ 民輝里  │ 

## 該村里前五個該商圈最多的 business_sub_type 

In [ ]:
def get_top5_subtype_data(district=None, village=None):
    # 動態構建 WHERE 條件
    conditions = []
    if district:
        conditions.append(f"district = '{district}'")
    if village:
        conditions.append(f"village = '{village}'")

    # 合成 WHERE 子句
    where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""
    
    res = con.sql(f"""--sql
            SELECT district, village, business_type, business_sub_type, COUNT(business_name) as shop_cnt, ROUND(AVG(capital)) as avg_capital
            FROM Business_Operation
            {where_clause}
            GROUP BY district, village, business_type, business_sub_type
            ORDER BY shop_cnt DESC
            LIMIT 5
          """)
    
    return res

get_village_data(district = '大安區', village = '民輝里')
# get_village_data(district = '大安區')
# get_village_data()

┌──────────┬─────────┬────────────────────────┬──────────────────────┬──────────┬─────────────┐
│ district │ village │     business_type      │  business_sub_type   │ shop_cnt │ avg_capital │
│ varchar  │ varchar │        varchar         │       varchar        │  int64   │   double    │
├──────────┼─────────┼────────────────────────┼──────────────────────┼──────────┼─────────────┤
│ 大安區   │ 民輝里  │ 住宿及餐飲業           │ 餐食業               │        6 │    126833.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 其他專賣零售業       │        5 │    150000.0 │
│ 大安區   │ 民輝里  │ 其他服務業             │ 美髮及美容美體業     │        4 │    150000.0 │
│ 大安區   │ 民輝里  │ 批發及零售業           │ 家用器具及用品零售業 │        4 │    150750.0 │
│ 大安區   │ 民輝里  │ 專業、科學及技術服務業 │ 廣告業               │        3 │    170000.0 │
└──────────┴─────────┴────────────────────────┴──────────────────────┴──────────┴─────────────┘

## 插入房東資料

In [105]:
def insert_representative(phone, name, is_agent=True):
    # 檢查是否已存在
    existing = con.sql(f"""--sql
    SELECT *
    FROM Representative 
    WHERE phone = '{phone}'
    """).df()

    # 如果不存在，則插入新記錄
    if existing.empty:
        con.sql(f"""--sql
        INSERT INTO Representative (phone, name, is_agent)
        VALUES ('{phone}', '{name}', {is_agent})
        """)
        return f"成功新增房東資料 : {phone}"
    else:
        return f"房東資料已存在"
    
# 範例
insert_representative('0912345678', 'test')

'房東資料已存在'

## 插入Shop_Rental_listing

In [147]:
def insert_shop_rental_listing(case_name, address, longitude, latitude, district, village, monthly_rent, deposit, area_ping, shop_floor,
                               total_floor, phone, is_available=True):
    existing = con.sql(f"""--sql
        SELECT *
        FROM Shop_Rental_Listing 
        WHERE case_name = '{case_name}' 
            AND address = '{address}' 
            AND longitude = '{longitude}' 
            AND latitude = '{latitude}' 
            AND district = '{district}' 
            AND village = '{village}' 
            AND monthly_rent = '{monthly_rent}' 
            AND deposit = '{deposit}' 
            AND shop_floor = '{shop_floor}' 
            AND total_floor = '{total_floor}' 
            AND phone = '{phone}'
    """).df()
    if existing.empty:
        con.sql(f"""--sql
        INSERT INTO Shop_Rental_Listing (
            case_id, case_name, address, longitude, latitude, district,
            village, monthly_rent, deposit, area_ping, shop_floor,
            total_floor, phone, is_available
        )
        VALUES (
            nextval('id_sequence'), '{case_name}', '{address}', {longitude}, {latitude},
            '{district}', '{village}', {monthly_rent}, {deposit}, {area_ping},
            {shop_floor}, {total_floor}, '{phone}', {is_available}
        )
        """)
        return f"成功新增一筆出租資訊"
    else:
        return f"此筆出租資訊已存在"

# 範例
insert_shop_rental_listing('test_case2', 'address', 0, 0, 'district', 'village', 0, 0, 0, '0', 0, '09')

'此筆出租資訊已存在'

In [161]:
con.sql("""--sql
    select *
    from Shop_Rental_Listing
    --where case_name = 'test_case'
""")

┌─────────┬───────────────────────────────────────────────────┬──────────────────────────┬─────────────────┬─────────────────┬──────────┬─────────┬──────────────┬─────────┬───────────┬────────────┬─────────────┬────────────┬──────────────┐
│ case_id │                     case_name                     │         address          │    longitude    │    latitude     │ district │ village │ monthly_rent │ deposit │ area_ping │ shop_floor │ total_floor │   phone    │ is_available │
│  int64  │                      varchar                      │         varchar          │     double      │     double      │ varchar  │ varchar │    int32     │  int32  │   int32   │  varchar   │    int32    │  varchar   │   boolean    │
├─────────┼───────────────────────────────────────────────────┼──────────────────────────┼─────────────────┼─────────────────┼──────────┼─────────┼──────────────┼─────────┼───────────┼────────────┼─────────────┼────────────┼──────────────┤
│       1 │ 士林夜市TOD1+2樓店面               

## 更新狀態為已出租

In [164]:
def mark_as_rented(case_name, address, phone):
    result = con.sql(f"""--sql
    UPDATE Shop_Rental_Listing
    SET is_available = FALSE
    WHERE case_name = '{case_name}' AND address = '{address}'AND phone = '{phone}';
""")  
    
    return f"成功將出租資訊 `{case_name}` 標記為已出租"

mark_as_rented('test_case', 'address', '09')

'成功將出租資訊 `test_case` 標記為已出租'